## Chromatic number problem code

In [25]:
from pysat.solvers import Solver

def var(i, color):
    return 3 * (i - 1) + {'R': 1, 'G': 2, 'Y': 3}[color]

def main():
    num_nodes = 7  # Update with the number of nodes in your graph
    edges = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7)]  # Update with the edges in your graph

    clauses = []

    # Encode the first part of the formula
    for i in range(1, num_nodes + 1):
        clauses.append([var(i, color) for color in ['R', 'G', 'Y']])
        clauses.append([-var(i, 'R'), -var(i, 'G')])
        clauses.append([-var(i, 'R'), -var(i, 'Y')])
        clauses.append([-var(i, 'G'), -var(i, 'Y')])

    # Add constraints that node 1 is red and node 2 is yellow
    clauses.append([var(1, 'R')])
    clauses.append([var(2, 'Y')])

    # Encode the second part of the formula (edge constraints)
    for edge in edges:
        i, j = edge
        for color in ['R', 'G', 'Y']:
            clauses.append([-var(i, color), -var(j, color)])

    # Use Solver to solve the CNF formula
    with Solver(name='m22', bootstrap_with=clauses) as m:
        is_sat = m.solve()

        if is_sat:
            model = m.get_model()
            assignment = [(i, color) for i in range(1, num_nodes + 1) for color in ['R', 'G', 'Y'] if model[var(i, color) - 1] > 0]
            print("Satisfiable solution found:")
            print(assignment)
        else:
            print("The formula is unsatisfiable.")

if __name__ == "__main__":
    main()

Satisfiable solution found:
[(1, 'R'), (2, 'Y'), (3, 'R'), (4, 'G'), (5, 'R'), (6, 'G'), (7, 'R')]


## Sudoko solver 

In [26]:
from pysat.solvers import Solver
def sudoko_solver(board):
    def var(i, j, no):
        return 100 * i + 10 * j + no
    
    clauses = []

    # Appending the given information
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                clauses.append([var(i+1, j+1, board[i][j])])

    # Each cell has exactly one number
    for i in range(1, 10):
        for j in range(1, 10):
            clauses.append([var(i, j, no) for no in range(1, 10)])
            for a in range(1, 10):
                for b in range(a + 1, 10):
                    clauses.append([-var(i, j, a), -var(i, j, b)])

    # Each number appears once in each row
    for i in range(1, 10):
        for no in range(1, 10):
            clauses.append([var(i, j, no) for j in range(1, 10)])
            for a in range(1, 10):
                for b in range(a + 1, 10):
                    clauses.append([-var(i, a, no), -var(i, b, no)])

    # Each number appears once in each column
    for j in range(1, 10):
        for no in range(1, 10):
            clauses.append([var(i, j, no) for i in range(1, 10)])
            for a in range(1, 10):
                for b in range(a + 1, 10):
                    clauses.append([-var(a, j, no), -var(b, j, no)])

    # Each number appears once in each 3x3 subgrid
    for no in range(1, 10):
        for a in range(3):
            for b in range(3):
                clauses.append([var(3*a + u, 3*b + v, no) for u in range(1, 4) for v in range(1, 4)])
                for u in range(1, 4):
                    for v in range(1, 4):
                        for i in range(u, 4):
                            for j in range(v if u == i else 1, 4):
                                if u != i or v != j:
                                    clauses.append([-var(3*a + u, 3*b + v, no), -var(3*a + i, 3*b + j, no)])

    # Use Solver to solve the CNF formula
    with Solver(name='m22', bootstrap_with=clauses) as m:
        is_sat = m.solve()

        if is_sat:
            model = m.get_model()
            for i in range(1, 10):
                for j in range(1, 10):
                    for no in range(1, 10):
                        if var(i, j, no) in model:
                            print(no, end=" ")
                print()
        else:
            print("The formula is unsatisfiable.")

board1 = [
        [3, 0, 6, 5, 0, 8, 4, 0, 0],
        [5, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 8, 7, 0, 0, 0, 0, 3, 1],
        [0, 0, 3, 0, 1, 0, 0, 8, 0],
        [9, 0, 0, 8, 6, 3, 0, 0, 5],
        [0, 5, 0, 0, 9, 0, 6, 0, 0],
        [1, 3, 0, 0, 0, 0, 2, 5, 0],
        [0, 0, 0, 0, 0, 0, 0, 7, 4],
        [0, 0, 5, 2, 0, 6, 3, 0, 0]
    ]

board2 = [
 [3, 0, 6, 0, 7, 0, 0, 9, 2],
 [5, 0, 0, 0, 0, 0, 7, 6, 8],
 [0, 8, 0, 0, 0, 9, 5, 3, 0],
 [2, 6, 0, 4, 0, 5, 9, 8, 0],
 [0, 0, 4, 8, 0, 3, 1, 0, 5],
 [8, 5, 0, 0, 0, 2, 6, 4, 3],
 [1, 3, 8, 0, 0, 7, 2, 0, 0],
 [0, 0, 2, 0, 0, 1, 0, 0, 0],
 [7, 4, 0, 0, 0, 0, 3, 0, 0]
]

sudoko_solver(board1)
print()
print()
sudoko_solver(board2)

3 1 6 5 7 8 4 9 2 
5 2 9 1 3 4 7 6 8 
4 8 7 6 2 9 5 3 1 
2 6 3 4 1 5 9 8 7 
9 7 4 8 6 3 1 2 5 
8 5 1 7 9 2 6 4 3 
1 3 8 9 4 7 2 5 6 
6 9 2 3 5 1 8 7 4 
7 4 5 2 8 6 3 1 9 


3 1 6 5 7 8 4 9 2 
5 2 9 1 3 4 7 6 8 
4 8 7 6 2 9 5 3 1 
2 6 3 4 1 5 9 8 7 
9 7 4 8 6 3 1 2 5 
8 5 1 7 9 2 6 4 3 
1 3 8 9 4 7 2 5 6 
6 9 2 3 5 1 8 7 4 
7 4 5 2 8 6 3 1 9 


## Sudoko generator

In [27]:
import random
from pysat.solvers import Solver
board = [
 [3, 1, 6, 5, 7, 8, 4, 9, 2],
 [5, 2, 9, 1, 3, 4, 7, 6, 8],
 [4, 8, 7, 6, 2, 9, 5, 3, 1],
 [2, 6, 3, 4, 1, 5, 9, 8, 7],
 [9, 7, 4, 8, 6, 3, 1, 2, 5],
 [8, 5, 1, 7, 9, 2, 6, 4, 3],
 [1, 3, 8, 9, 4, 7, 2, 5, 6],
 [6, 9, 2, 3, 5, 1, 8, 7, 4],
 [7, 4, 5, 2, 8, 6, 3, 1, 9]
]

def var(i, j, no):
    return 100 * i + 10 * j + no

def is_unique(board):
    clauses = []

    # Appending the given information
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                clauses.append([var(i+1, j+1, board[i][j])])

    # Each cell has exactly one number
    for i in range(1, 10):
        for j in range(1, 10):
            clauses.append([var(i, j, no) for no in range(1, 10)])
            for a in range(1, 10):
                for b in range(a + 1, 10):
                    clauses.append([-var(i, j, a), -var(i, j, b)])

    # Each number appears once in each row
    for i in range(1, 10):
        for no in range(1, 10):
            clauses.append([var(i, j, no) for j in range(1, 10)])
            for a in range(1, 10):
                for b in range(a + 1, 10):
                    clauses.append([-var(i, a, no), -var(i, b, no)])

    # Each number appears once in each column
    for j in range(1, 10):
        for no in range(1, 10):
            clauses.append([var(i, j, no) for i in range(1, 10)])
            for a in range(1, 10):
                for b in range(a + 1, 10):
                    clauses.append([-var(a, j, no), -var(b, j, no)])

    # Each number appears once in each 3x3 subgrid
    for no in range(1, 10):
        for a in range(3):
            for b in range(3):
                clauses.append([var(3*a + u, 3*b + v, no) for u in range(1, 4) for v in range(1, 4)])
                for u in range(1, 4):
                    for v in range(1, 4):
                        for i in range(u, 4):
                            for j in range(v if u == i else 1, 4):
                                if u != i or v != j:
                                    clauses.append([-var(3*a + u, 3*b + v, no), -var(3*a + i, 3*b + j, no)])

    # Use Solver to solve the CNF formula
    with Solver(name='m22', bootstrap_with=clauses) as m:
        is_sat = m.solve()

        if is_sat:
            model = m.get_model()
            sol1 = [[0 for _ in range(9)] for _ in range(9)]
            for i in range(0, 9):
                for j in range(0, 9):
                    for no in range(1, 10):
                        if var(i+1, j+1, no) in model:
                            sol1[i][j] = no
            
            temp = []
            for i in range(0,9):      
                for j in range(0,9):
                    temp.append(-var(i+1, j+1, sol1[i][j]))
            
            clauses.append(temp)
            #print(temp)
            
            with Solver(name='m22', bootstrap_with=clauses) as m:
                is_sat = m.solve()
                if is_sat:
                    return False
                else:
                    return True
            
        else:
            return False
    
N = 0

while(True):
    #Generate random integer from 0,1,2,3,4,5,6,7,8
    i = random.randint(0, 8)
    j = random.randint(0, 8)
    temp = board[i][j]
    board[i][j] = 0
    if(is_unique(board)):
        continue
    else:
        board[i][j] = temp
        break


for i in range(1, 10):
    for j in range(1, 10):
        print(board[i-1][j-1], end=" ")
    print()



3 0 6 0 0 0 0 0 0 
5 2 0 0 3 0 7 0 8 
4 0 0 6 2 0 5 3 1 
2 0 3 4 0 5 0 0 0 
0 0 0 0 6 0 1 0 0 
0 5 1 7 0 0 6 0 3 
1 3 0 0 4 0 0 5 6 
0 0 2 0 5 0 8 7 0 
0 4 0 2 8 0 0 0 0 
